In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import time
import glob

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/1")
height, width = hub.get_expected_image_size(module)

In [0]:
print("height: {0}, width: {1}".format(height, width))

In [0]:
car_image_files = glob.glob("drive/My Drive/xylosai/tensorflow/images/car/*.jpg")
len(car_image_files)

# With Dataset pipeline API

In [0]:
from tensorflow.data import Dataset

In [0]:
def _parse_function(filename):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string,channels=3)
  image_resized = tf.image.resize_images(image_decoded, [height, width])
  return image_resized

In [0]:
dataset = Dataset.from_tensor_slices(car_image_files[0:10]).map(_parse_function).map(lambda x: tf.div(x,255)).map(lambda x: tf.expand_dims(x, axis=0))

In [0]:
dataset.output_shapes

In [0]:
iterator = dataset.make_one_shot_iterator()

In [0]:
start_time = time.time()
with tf.device('/cpu:0'):
  next_image = iterator.get_next()
  for i in range(10):
    try:
      print("image {0}".format(i))
      print(module(next_image))

    except (tf.errors.OutOfRangeError, KeyboardInterrupt):
      break
      
duration = time.time() - start_time
print("Inference took {0} seconds".format(duration))